<a href="https://colab.research.google.com/github/tset-tset-tset/tset-tset-tset.github.io/blob/master/JSUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!gdown --id 1f7bIQfwWdFOxeaYzs5Cw-HTcA8uwQ8qp -O jsut_ver1.1.zip
!unzip -q jsut_ver1.1.zip

replace jsut_ver1.1/precedent130/transcript_utf8.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [15]:
## for mecab & mecab-ipadic-neologd
!sudo apt-get install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!cd mecab-ipadic-neologd; sudo ./bin/install-mecab-ipadic-neologd -n -a -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
xz-utils is already the newest version (5.2.2-1.3).
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
curl is already the newest version (7.58.0-2ubuntu3.9).
file is already the newest version (1:5.32-2ubuntu0.4).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[ins

In [33]:
## for uconv
!sudo apt-get install icu-devtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
icu-devtools is already the newest version (60.2-3ubuntu3.1).
icu-devtools set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [77]:
%%bash

cd /content/jsut_ver1.1/

rm -f metadata.csv
rm -f metadata_kana.csv
mkdir -p wavs

function convertjsut(){
    TARGET=$1
    cat ${TARGET}/transcript_utf8.txt | while IFS=: read id text; 
    do
        echo -n "${id}|" >> metadata_kana.csv
        cp ${TARGET}/wav/${id}.wav wavs/${id}.wav
        echo "${text}" | mecab -Odump -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd > /tmp/${id}.tmp
        cat /tmp/${id}.tmp | perl -lne 'next if not m{\d+ (\S+) ([^,]+),(?:[^,]+,){4,6}([^,]+),}; $add = ($3 eq "*" ? $1 : $3); if ($2 eq "助詞" or $2 eq "助動詞") { $f = 1; $o .= $add; } else { if ($f == 1) { $r .= " $o"; $o = ""; $f = 0; } if ($1 ne "BOS" and $1 ne "EOS") { $o .= $add; } } END { $o = "$r $o"; $o =~ s{^\s*(.*?)\s*$}{\1}g; $o =~ s{ ?、}{, }g; $o =~ s{ ?。}{.}g; print $o }' >> metadata_kana.csv
        rm /tmp/${id}.tmp
    done
    ## 長音とくっついて1字になってしまうので最初に長音を変換する…
    cat metadata_kana.csv | perl -pe 's{ー}{-}g;' | uconv -f UTF-8 -t UTF-8 -x Latin | perl -pe 's{n'\''}{nn}g; s{i~}{}g;' > metadata.csv
}

convertjsut basic5000
convertjsut travel1000

echo [metadata.csv]
shuf metadata.csv | head
echo
echo [metadata_kana.csv]
shuf metadata_kana.csv | head

[metadata.csv]
TRAVEL1000_0607|shoruda-bagguha motte kaerimasu.
BASIC5000_0445|maasoukoufunshinaide, osuwarikudasai.
BASIC5000_3578|konotokuchoutekina sotoseikino yousukara, kagakutekikenkyuuga susumumaeniha, shiyuudoutaino katouna seibutsuto kangaerarete ita jikimo atta youdearu.
TRAVEL1000_0764|konbanha, tsuinha subetemanshitsudesu.
BASIC5000_4941|mateba, kairono, hiyoriari.
BASIC5000_1315|ogawaga sarasarato nagareruotoga kikoeru.
BASIC5000_0061|mirutonnno sakuhinwo yonda kotoga arimasuka.
BASIC5000_2298|nabeni gomaaburawo chuubide nesshi, ninnniku, keinikuwo irete itame, iroga kawattara, chuukasu-puto hakusaiwo irete niru.
BASIC5000_2696|jinkounaijiha, gijyutsutekini koumyouna kikidesu.
BASIC5000_2758|gakugyouseisekiwo ageruniha, isshoukenmeibenkyousuruhitsuyouga aru.

[metadata_kana.csv]
BASIC5000_2900|ナマイキニモ, カレハ ワタシノ モウシデヲ コトワッタ.
BASIC5000_2392|ドタンバニ ナッテ, コトバガ デテ コナカッタ.
BASIC5000_3276|キンノ タメニ シガンシタト イウガ, タノ パイロットノ ヒッシニ タタカウスガタヲ メニ シテ, シダイニタイドヲ アラタメル.
BASIC5000_4253|ムカシ, シンカクサイボウガ